In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

visual_taxonomy_path = kagglehub.competition_download('visual-taxonomy')

print('Data source import complete.')


100%|██████████| 3.52G/3.52G [02:55<00:00, 21.5MB/s]

Extracting files...


Data source import complete.


In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [27]:
df1 = pd.read_csv(visual_taxonomy_path+"/train.csv")
df2 = pd.read_csv(visual_taxonomy_path+"/test.csv")
sample = pd.read_csv(visual_taxonomy_path+'/sample_submission.csv')
df1["id"] = df1["id"].astype(str).str.zfill(6)
df2["id"] = df2["id"].astype(str).str.zfill(6)

In [28]:
df1['image_path'] = df1['id'].apply(lambda x: visual_taxonomy_path+f"/train_images/{str(x).zfill(6)}.jpg")

In [29]:
# Calculate the number of rows to remove based on the condition
rem_row = df1.index[df1.apply(lambda row: row.isna().sum() > 16 - row['len'], axis=1)].tolist()

In [30]:
# rem_row = []
# for i in range(df1.shape[0]):
#     if df1.iloc[i,:].isna().sum() > 11 - df1.loc[i,'len']:
#         rem_row.append(i)

In [31]:
df1.drop(rem_row, axis = 0, inplace = True)
df1.reset_index(drop= True, inplace = True)

In [32]:
# for i in range(df1.shape[0]):
#     if df1.iloc[i,:].isna().sum() == 10 - df1.loc[i,'len']:
#         df1.iloc[i,:] = df1.iloc[i,:].fillna("extra")

In [33]:
df1

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path
0,000000,Men Tshirts,5,default,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN,/root/.cache/kagglehub/competitions/visual-tax...
1,000001,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/root/.cache/kagglehub/competitions/visual-tax...
2,000002,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/root/.cache/kagglehub/competitions/visual-tax...
3,000003,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/root/.cache/kagglehub/competitions/visual-tax...
4,000004,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/root/.cache/kagglehub/competitions/visual-tax...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64300,070373,Women Tops & Tunics,10,blue,regular,regular,round neck,casual,solid,solid,short sleeves,default,knitted,/root/.cache/kagglehub/competitions/visual-tax...
64301,070374,Women Tops & Tunics,10,multicolor,fitted,regular,square neck,casual,printed,default,short sleeves,regular sleeves,ruffles,/root/.cache/kagglehub/competitions/visual-tax...
64302,070375,Women Tops & Tunics,10,yellow,regular,crop,round neck,casual,default,default,short sleeves,regular sleeves,knitted,/root/.cache/kagglehub/competitions/visual-tax...
64303,070376,Women Tops & Tunics,10,maroon,fitted,crop,round neck,casual,solid,solid,short sleeves,regular sleeves,knitted,/root/.cache/kagglehub/competitions/visual-tax...


In [34]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder

def apply_knn_imputer_categorical(df_temp, n_neighbors=5):

    max_len = df_temp['len'].max()
    useful_columns = [f'attr_{i}' for i in range(1, max_len + 1)]

    encoder = OrdinalEncoder()
    df_encoded = df_temp.copy()
#     print(df_temp[useful_columns])
    df_encoded[useful_columns] = encoder.fit_transform(df_temp[useful_columns])
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed_array = imputer.fit_transform(df_encoded[useful_columns])

    df_imputed = pd.DataFrame(df_imputed_array, columns=useful_columns, index=df_temp.index)
#     print(df_imputed)

    df_imputed[useful_columns] = df_imputed[useful_columns].round().astype(int)
#     print(df_imputed)

    df_imputed[useful_columns] = encoder.inverse_transform(df_imputed[useful_columns])
#     print(df_imputed)
    cols = [x for x in df_temp.columns.tolist() if x not in useful_columns]
    df_imputed = pd.concat([df_imputed, df_temp[cols]], axis = 1)
    df_imputed = df_imputed[df_temp.columns]
#     print(df_imputed)

    return df_imputed


In [35]:
# Create a copy of the original DataFrame to work on
df_imputed = df1.copy()

# Get all unique categories in the dataset
categories = df_imputed['Category'].unique()

# Loop over each category and apply the column transformer function
for category in categories:
    # Step 1: Filter the DataFrame for the current category
    df_temp = df_imputed[df_imputed['Category'] == category].copy()

    # Step 2: Apply the column transformer to this category's DataFrame
    df_temp_imputed = apply_knn_imputer_categorical(df_temp)

    # Step 3: Update the original DataFrame with the imputed values
    # Determine the useful columns for this category
    useful_columns = [f'attr_{i}' for i in range(1, df_temp['len'].max() + 1)]

    # Assign the imputed values back to the original DataFrame
    df_imputed.loc[df_temp.index, useful_columns] = df_temp_imputed[useful_columns]

# Now, df_imputed contains the imputed values for all categories

In [36]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Dictionary to hold encoders and data for each category
encoders = {}
category_dfs_encoded = {}

# Function to one-hot encode the attributes and store the encoder
def one_hot_encode_category(df, category, num_attributes):
    # Initialize OneHotEncoder
    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    # Select the useful columns (attributes)
    useful_columns = [f'attr_{i}' for i in range(1, num_attributes + 1)]

    # Fit and transform the attributes
    ohe_encoded = ohe.fit_transform(df[useful_columns])

    # Store the encoder for later decoding
    encoders[category] = ohe

    # Combine the encoded attributes with id, Category, and image_path
    one_hot_df = pd.concat([df[['id', 'Category', 'image_path']].reset_index(drop=True),
                            pd.DataFrame(ohe_encoded, columns=ohe.get_feature_names_out())], axis=1)

    return one_hot_df

# Split the data by category and apply one-hot encoding
category_attributes = {
    'Men Tshirts': 5,
    'Sarees': 10,
    'Kurtis': 9,
    'Women Tshirts': 8,
    'Women Tops & Tunics': 10
}

for category, num_attributes in category_attributes.items():
    # Filter the DataFrame by category
    df_temp = df_imputed[df_imputed['Category'] == category].copy()

    # Apply one-hot encoding
    df_encoded = one_hot_encode_category(df_temp, category, num_attributes)

    # Store the one-hot encoded DataFrame in a dictionary
    category_dfs_encoded[category] = df_encoded

# Example: Access the one-hot encoded DataFrames for each category
df_men_tshirts_encoded = category_dfs_encoded['Men Tshirts']
df_sarees_encoded = category_dfs_encoded['Sarees']
df_kurtis_encoded = category_dfs_encoded['Kurtis']
df_women_tshirts_encoded = category_dfs_encoded['Women Tshirts']
df_women_tunics_encoded = category_dfs_encoded['Women Tops & Tunics']

# Print the shape of the one-hot encoded DataFrame for one category
print(f"Shape of Men Tshirts encoded data: {df_men_tshirts_encoded.shape}")
print(f"Shape of Sarees encoded data: {df_sarees_encoded.shape}")
print(f"Shape of Kurtis encoded data: {df_kurtis_encoded.shape}")
print(f"Shape of Women Tshirts encoded data: {df_women_tshirts_encoded.shape}")
print(f"Shape of Women Tunics encoded data: {df_women_tunics_encoded.shape}")

# Display the first few rows of the encoded data for "Men Tshirts"
print(df_kurtis_encoded.head())

Shape of Men Tshirts encoded data: (7267, 16)
Shape of Sarees encoded data: (17727, 51)
Shape of Kurtis encoded data: (6803, 33)
Shape of Women Tshirts encoded data: (17922, 32)
Shape of Women Tunics encoded data: (14586, 53)
       id Category                                         image_path  \
0  025778   Kurtis  /root/.cache/kagglehub/competitions/visual-tax...   
1  025779   Kurtis  /root/.cache/kagglehub/competitions/visual-tax...   
2  025780   Kurtis  /root/.cache/kagglehub/competitions/visual-tax...   
3  025781   Kurtis  /root/.cache/kagglehub/competitions/visual-tax...   
4  025782   Kurtis  /root/.cache/kagglehub/competitions/visual-tax...   

   attr_1_black  attr_1_blue  attr_1_green  attr_1_grey  attr_1_maroon  \
0           1.0          0.0           0.0          0.0            0.0   
1           0.0          0.0           0.0          0.0            0.0   
2           0.0          0.0           0.0          0.0            0.0   
3           0.0          0.0           

In [37]:
category_dfs_encoded['Men Tshirts']

,id,Category,image_path,attr_1_black,attr_1_default,attr_1_multicolor,attr_1_white,attr_2_polo,attr_2_round,attr_3_printed,attr_3_solid,attr_4_default,attr_4_solid,attr_4_typography,attr_5_long sleeves,attr_5_short sleeves
0,000000,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,000001,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,000002,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,000003,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,000004,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,007424,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
7263,007425,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
7264,007426,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
7265,007429,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [38]:
df2['image_path'] = df2['id'].apply(lambda x: visual_taxonomy_path+f"/test_images/{str(x).zfill(6)}.jpg")
df2

,id,Category,image_path
0,000000,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...
1,000001,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...
2,000002,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...
3,000003,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...
4,000004,Men Tshirts,/root/.cache/kagglehub/competitions/visual-tax...
...,...,...,...
30200,030484,Women Tops & Tunics,/root/.cache/kagglehub/competitions/visual-tax...
30201,030485,Women Tops & Tunics,/root/.cache/kagglehub/competitions/visual-tax...
30202,030486,Women Tops & Tunics,/root/.cache/kagglehub/competitions/visual-tax...
30203,030487,Women Tops & Tunics,/root/.cache/kagglehub/competitions/visual-tax...


In [39]:
test_df = df2.copy()

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import numpy as np
import os

# Set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom dataset class for image classification
class ProductDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = row['image_path']
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        labels = row.iloc[3:].values.astype(float)  # Adjust according to your DataFrame structure
        return image, torch.tensor(labels, dtype=torch.float32)

# Define transformations
def get_transform():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

# Prepare dataset
def prepare_dataset(df):
    return ProductDataset(df, transform=get_transform())

# Initialize the ResNet model for multi-label classification
def create_resnet_model(num_labels):
    model = models.resnet50(pretrained=True)  # Load ResNet50 with pretrained weights
    # Modify the final layer for multi-label classification
    model.fc = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(model.fc.in_features, num_labels)
    )
    return model

# Training function
def train_resnet_model(model, dataloader, num_epochs=1):
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()  # Suitable for multi-label classification
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            running_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")

    return model

# Generalized function for creating and training the ResNet model for any category
def create_and_train_resnet_model_for_category(df_encoded, category_name, num_epochs=3):
    num_labels = df_encoded.shape[1] - 3  # Calculate based on columns in df_encoded
    resnet_model = create_resnet_model(num_labels)
    train_dataset = prepare_dataset(df_encoded)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    trained_model = train_resnet_model(resnet_model, train_loader, num_epochs)

    # Save the trained model
    model_filename = f'resnet_model_{category_name}.pth'
    torch.save(trained_model.state_dict(), model_filename)
    print(f"Model for {category_name} saved as {model_filename}")

    return trained_model

# Function to test the model for a given category
def test_resnet_model(df_test, category_name, num_labels):
    test_dataset = prepare_dataset(df_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Load the trained ResNet model
    model = create_resnet_model(num_labels)
    model.load_state_dict(torch.load(f'resnet_model_{category_name}.pth'))
    model = model.to(device)
    model.eval()

    # Make predictions
    predictions = []
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities
            predictions.append(preds.cpu().numpy())

    return np.concatenate(predictions, axis=0)

# Decode predictions using the OneHotEncoder dynamically based on the category and number of attributes
def decode_predictions(predictions, encoder):
    decoded_predictions = []

    for pred in predictions:
        decoded = []
        val = 0
        for i in range(len(encoder.categories_)):
            attr_probs = pred[val: val + len(encoder.categories_[i])]
            val = val + len(encoder.categories_[i])
            max_index = np.argmax(attr_probs)
            decoded.append(encoder.categories_[i][max_index])

        decoded_predictions.append(decoded)

    return np.array(decoded_predictions)

# Prepare submission
def prepare_submission(df_test, decoded_attributes, category_name, num_attributes):
    submission_data = {
        'id': df_test['id'].values
    }

    actual_num_attributes = decoded_attributes.shape[1]
    for i in range(1, actual_num_attributes + 1):
        submission_data[f'attr_{i}'] = decoded_attributes[:, i - 1]

    submission_df = pd.DataFrame(submission_data)
    submission_filename = f"{category_name}_predictions.csv"
    submission_df.to_csv(submission_filename, index=False)
    print(f"Submission file saved: {submission_filename}")

# Generalized function to handle decoding and submission for any category
def decode_and_save_submission(predictions, df_test, encoder, category_name, num_attributes):
    pred_array = predictions
    decoded_attributes = decode_predictions(pred_array, encoder)
    submission_df = prepare_submission(df_test, decoded_attributes, category_name, num_attributes)

    submission_file = f'{category_name}_predictions.csv'
    submission_df.to_csv(submission_file, index=False)
    print(f"Submission file for {category_name} saved as {submission_file}")
    FileLink(f'/content/{category_name}_predictions.csv')

# Function to save the trained model for a specific category
def save_model(trained_model, category_name):
    model_filename = f'resnet_model_{category_name}.pth'
    torch.save(trained_model.state_dict(), model_filename)
    print(f"Model for {category_name} saved as {model_filename}")

# Define pipeline for multiple categories
categories = ['Sarees', 'Kurtis', 'Women Tshirts', 'Women Tops & Tunics', 'Men Tshirts']

In [48]:
def pipeline(categories, category_dfs_encoded, test_df, num_epoch=3):
    for category in categories:
        print(f"\nProcessing category: {category}")

        df_encoded = category_dfs_encoded[category]

        # Create and train the model for this category
        num_labels = df_encoded.shape[1] - 3
        trained_model = create_and_train_resnet_model_for_category(df_encoded, category, num_epoch)

        # Save the trained model
        save_model(trained_model, category)

        # Prepare the test data for this category
        df_test_category = test_df[test_df['Category'] == category].copy()
        df_test_category['image_path'] = df_test_category['id'].apply(lambda x: visual_taxonomy_path+f"/test_images/{str(x).zfill(6)}.jpg")


        # Test the model on the test data
        predictions = test_resnet_model(df_test_category, category, num_labels)

        # Decode predictions and prepare submission
        decoded_predictions = decode_predictions(predictions, encoders[category])
        prepare_submission(df_test_category, decoded_predictions, category, num_labels)

In [ ]:
pipeline(categories, category_dfs_encoded, test_df, num_epoch = 10)


Processing category: Sarees


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.2571
Epoch [2/10], Loss: 0.2376
Epoch [3/10], Loss: 0.2336
Epoch [4/10], Loss: 0.2312
Epoch [5/10], Loss: 0.2294
Epoch [6/10], Loss: 0.2277
Epoch [7/10], Loss: 0.2264
Epoch [8/10], Loss: 0.2261
Epoch [9/10], Loss: 0.2245
Epoch [10/10], Loss: 0.2234
Model for Sarees saved as resnet_model_Sarees.pth
Model for Sarees saved as resnet_model_Sarees.pth


<ipython-input-47-fd8aed4d88b6>:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'resnet_model_{category_name}.pth'))


Submission file saved: Sarees_predictions.csv

Processing category: Kurtis
Epoch [1/10], Loss: 0.2655
Epoch [2/10], Loss: 0.1960
Epoch [3/10], Loss: 0.1762
Epoch [4/10], Loss: 0.1633
Epoch [5/10], Loss: 0.1523
Epoch [6/10], Loss: 0.1438
Epoch [7/10], Loss: 0.1362
Epoch [8/10], Loss: 0.1281
Epoch [9/10], Loss: 0.1211
Epoch [10/10], Loss: 0.1159
Model for Kurtis saved as resnet_model_Kurtis.pth
Model for Kurtis saved as resnet_model_Kurtis.pth
Submission file saved: Kurtis_predictions.csv

Processing category: Women Tshirts


In [ ]:
alldf = []
for i in categories:
    df = pd.read_csv(f'/content/{i}_predictions.csv')
    alldf.append(df)

In [ ]:
df_final = pd.concat(alldf, axis = 0)
df_final = df_final.sort_values(by = 'id')
df_final.reset_index(drop= True, inplace = True)

In [ ]:
df_final = pd.concat([df_final, pd.read_csv(visual_taxonomy_path+"/test.csv")['Category']], axis = 1)

In [ ]:
df_final['len'] = df_final['Category'].map(category_attributes)
df_final = df_final[sample.columns]
df_final.fillna('extra', inplace = True)

In [ ]:
df_final.to_csv('meesho_submission21_10ep_knn5_6na_resnet.csv')

In [ ]:
df_final

In [ ]:
# pd.read_csv('/kaggle/working/meesho_submission16_10ep_knn5_3na_resnet.csv')

,Unnamed: 0,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,0,Men Tshirts,5,multicolor,round,printed,typography,long sleeves,extra,extra,extra,extra,extra
1,1,1,Men Tshirts,5,white,round,printed,typography,long sleeves,extra,extra,extra,extra,extra
2,2,2,Men Tshirts,5,white,round,printed,default,short sleeves,extra,extra,extra,extra,extra
3,3,3,Men Tshirts,5,default,polo,solid,solid,short sleeves,extra,extra,extra,extra,extra
4,4,4,Men Tshirts,5,black,round,solid,solid,long sleeves,extra,extra,extra,extra,extra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30200,30200,30484,Women Tops & Tunics,10,green,boxy,crop,round neck,casual,printed,typography,short sleeves,regular sleeves,ruffles
30201,30201,30485,Women Tops & Tunics,10,default,regular,regular,round neck,casual,solid,solid,short sleeves,puff sleeves,knitted
30202,30202,30486,Women Tops & Tunics,10,default,regular,regular,v-neck,casual,solid,solid,short sleeves,puff sleeves,knitted
30203,30203,30487,Women Tops & Tunics,10,multicolor,regular,regular,round neck,casual,default,default,short sleeves,regular sleeves,knitted


In [43]:
# %cd /kaggle/working
from IPython.display import FileLink

In [ ]:
%cd ..

/kaggle


In [ ]:
%cd /kaggle/working

/kaggle/working


In [ ]:
from IPython.display import FileLink

In [46]:
FileLink('/content/meesho_submission20_10ep_knn5_4na_resnet.csv')

/content/meesho_submission20_10ep_knn5_4na_resnet.csv

In [ ]:
# import torch
# from transformers import ViTForImageClassification, ViTFeatureExtractor
# from torch.utils.data import DataLoader
# from PIL import Image
# import pandas as pd
# import numpy as np
# import os
# import torch.optim as optim
# import torch.nn as nn

# # Load the feature extractor
# feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# # Set device (GPU or CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Custom dataset class for ViT
# class ProductDataset(torch.utils.data.Dataset):
#     def __init__(self, dataframe, transform=None):
#         self.dataframe = dataframe
#         self.transform = transform

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, idx):
#         row = self.dataframe.iloc[idx]
#         image_path = row['image_path']
#         image = Image.open(image_path).convert('RGB')
#         if self.transform:
#             image = self.transform(image)
#         labels = row.iloc[3:].values.astype(float)  # Starting from 4th column onwards (attributes)
#         return image, torch.tensor(labels, dtype=torch.float32)

# # Apply ViT feature extractor transformations to the dataset
# def vit_transform(image):
#     return feature_extractor(images=image, return_tensors="pt").pixel_values[0]

# # Prepare dataset
# def prepare_dataset(df):
#     return ProductDataset(df, transform=vit_transform)

# # Initialize the ViT model for multi-label classification
# def create_vit_model(num_labels):
#     model = ViTForImageClassification.from_pretrained(
#         'google/vit-base-patch16-224-in21k',
#         num_labels=num_labels
#     )
#     model.classifier = nn.Sequential(
#         nn.Dropout(p=0.3),
#         nn.Linear(model.config.hidden_size, num_labels)
#     )
#     return model

# def train_vit_model(model, dataloader, num_epochs=1):
#     model = model.to(device)
#     criterion = nn.BCEWithLogitsLoss()  # Suitable for multi-label classification
#     optimizer = optim.AdamW(model.parameters(), lr=1e-4)

#     model.train()
#     for epoch in range(num_epochs):
#         running_loss = 0.0
#         for images, labels in dataloader:
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(images).logits  # Forward pass
#             loss = criterion(outputs, labels)
#             loss.backward()  # Backpropagation
#             optimizer.step()  # Update weights
#             running_loss += loss.item()
#         print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")

#     return model

# # Generalized function for creating and training the ViT model for any category
# def create_and_train_vit_model_for_category(df_encoded, category_name, num_epochs=3):
#     # Calculate the number of labels based on the encoded dataframe
#     num_labels = df_encoded.shape[1] - 3  # Subtract 'id', 'Category', 'image_path'

#     # Create the ViT model
#     vit_model = create_vit_model(num_labels)

#     # Prepare dataset and DataLoader
#     train_dataset = prepare_dataset(df_encoded)
#     train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

#     # Train the model
#     trained_model = train_vit_model(vit_model, train_loader, num_epochs)

#     # Save the trained model
#     model_filename = f'vit_model_{category_name}.pth'
#     torch.save(trained_model.state_dict(), model_filename)
#     print(f"Model for {category_name} saved as {model_filename}")

#     return trained_model

# # Function to load and test the model for a given category
# def test_vit_model(df_test, category_name, num_labels):
#     # Prepare test dataset
#     test_dataset = prepare_dataset(df_test)
#     test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#     # Load the trained model
#     model = create_vit_model(num_labels)
#     model.load_state_dict(torch.load(f'vit_model_{category_name}.pth'))
#     model = model.to(device)
#     model.eval()

#     # Make predictions
#     predictions = []
#     with torch.no_grad():
#         for images, ids in test_loader:
#             images = images.to(device)
#             outputs = model(images).logits
#             preds = torch.sigmoid(outputs)
#             predictions.append(preds.cpu().numpy())

#     return np.concatenate(predictions, axis=0)

# # Decode predictions using the OneHotEncoder dynamically based on the category and number of attributes
# def decode_predictions(predictions, encoder):
#     decoded_predictions = []

#     for pred in predictions:
#         decoded = []
#         val = 0
#         for i in range(len(encoder.categories_)):
#             # Get the probabilities for the current attribute (i-th group of one-hot encoded columns)
#             attr_probs = pred[val : val + len(encoder.categories_[i])]
#             val = val + len(encoder.categories_[i])

#             # Ensure that we pick the category with the highest probability, even if all are below the threshold
#             max_index = np.argmax(attr_probs)  # Find the index of the highest probability

#             decoded.append(encoder.categories_[i][max_index])  # Use the encoder to map back the category

#         decoded_predictions.append(decoded)

#     return np.array(decoded_predictions)

# def prepare_submission(df_test, decoded_attributes, category_name, num_attributes):
#     # Prepare the submission data
#     submission_data = {
#         'id': df_test['id'].values  # Include the 'id' column
#     }

#     # Get the actual number of decoded attributes from the shape of decoded_attributes
#     actual_num_attributes = decoded_attributes.shape[1]

#     # Dynamically add attribute columns to the dictionary based on the actual number of decoded attributes
#     for i in range(1, actual_num_attributes + 1):
#         submission_data[f'attr_{i}'] = decoded_attributes[:, i - 1]  # Adjust for zero-indexing in arrays

#     # Convert the dictionary to a DataFrame
#     submission_df = pd.DataFrame(submission_data)

#     # Save the submission DataFrame to a CSV file
#     submission_filename = f"{category_name}_predictions.csv"
#     submission_df.to_csv(submission_filename, index=False)
#     print(f"Submission file saved: {submission_filename}")


# # Generalized function to handle decoding and submission for any category
# def decode_and_save_submission(predictions, df_test, encoder, category_name, num_attributes):
#     # Convert the predictions array
#     pred_array = predictions

#     # Decode the predictions using the stored OneHotEncoder
#     decoded_attributes = decode_predictions(pred_array, encoder)

#     # Prepare the submission DataFrame
#     submission_df = prepare_submission(df_test, decoded_attributes, num_attributes)

#     # Save the submission DataFrame to CSV
#     submission_file = f'{category_name}_predictions.csv'
#     submission_df.to_csv(submission_file, index=False)

#     print(f"Submission file for {category_name} saved as {submission_file}")

#     return submission_df

# # Function to save the trained model for a specific category
# def save_model(trained_model, category_name):
#     model_filename = f'vit_model_{category_name}.pth'
#     torch.save(trained_model.state_dict(), model_filename)
#     print(f"Model for {category_name} saved as {model_filename}")

# categories = ['Sarees', 'Kurtis', 'Women Tshirts', 'Women Tops & Tunics', 'Men Tshirts']

# def pipeline(categories, category_dfs_encoded, test_df , num_epoch = 3):
#     # Loop over each category provided in the input
#     for category in categories:
#         print(f"\nProcessing category: {category}")

#         # 1. Prepare the dataset and dataloader for the category
#         df_encoded = category_dfs_encoded[category]  # Get the encoded dataframe for the category

#         # 2. Create and train the model for this category
#         num_labels = df_encoded.shape[1] - 3  # Subtract 'id', 'Category', 'image_path'
#         trained_model = create_and_train_vit_model_for_category(df_encoded ,category,num_epoch)

#         # 3. Save the trained model
#         save_model(trained_model, category)

#         # 4. Prepare the test data for this category
#         df_test_category = test_df[test_df['Category'] == category].copy()
#         df_test_category['image_path'] = df_test_category['id'].apply(lambda x: f"/kaggle/input/visual-taxonomy/test_images/{x}.jpg")
#         test_dataset = prepare_dataset(df_test_category)  # Prepare the test dataset
#         test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#         # 5. Test the model on the test data
#         predictions = test_vit_model(df_test_category, category, num_labels)  # Get predictions
#         print(pd.DataFrame(predictions))

#         # 6. Decode the predictions using the stored OneHotEncoder
#         pred_array = np.concatenate(predictions, axis=0)  # Concatenate along rows
#         pred_array = pred_array.reshape(-1, num_labels)  # Reshape to (samples, num_labels)

#         decoded_predictions = decode_predictions(pred_array, encoders[category])
#         print(pd.DataFrame(decoded_predictions))


#         # 7. Prepare and save the submission for this category
#         prepare_submission(df_test_category, decoded_predictions, category, num_labels)